In [ ]:
%matplotlib inline
import os
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import keras
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from keras.models import load_model
import copy
import PIL.Image
# some plotting defaults
import seaborn as sns
sns.set_style('whitegrid', {'axes.grid': False})
SSIZE=10
MSIZE=12
BSIZE=14
plt.rc('font', size=SSIZE)
plt.rc('axes', titlesize=MSIZE)
plt.rc('axes', labelsize=MSIZE)
plt.rc('xtick', labelsize=MSIZE)
plt.rc('ytick', labelsize=MSIZE)
plt.rc('legend', fontsize=MSIZE)
plt.rc('figure', titlesize=MSIZE)
plt.rcParams['font.family'] = "sans-serif"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Some Preprocessing Functions

In [ ]:
def load_image(file_path, resize=True,
               sztple=(224, 224),
               normalize=None):
    """Simple load image function."""
    img = PIL.Image.open(file_path).convert('RGB')
    if resize:
        img = img.resize(sztple, PIL.Image.ANTIALIAS)
    img = np.asarray(img)
    if normalize:
        img = normalize(img)
    return img


def normalize_image(x):
    """Normalize the img."""
    x = np.array(x).astype(np.float32)
    x_min = x.min()
    x_max = x.max()
    x_norm = (x - x_min) / (x_max - x_min)
    return x_norm


def grayscalenorm(img, percentile=99):
    """Normalize to [0, 1]."""
    assert len(img.shape) == 3

    img2d = np.sum(np.abs(img), axis=2)
    vmax = np.percentile(img2d, percentile)
    vmin = np.min(img2d)
    return np.clip((img2d - vmin) / (vmax - vmin), 0, 1)


def posnegnorm(img, percentile=99):
    """Normalize to [-1, 1]."""
    assert len(img.shape) == 3

    img2d = np.sum(img, axis=2)
    span = abs(np.percentile(img2d, percentile))
    vmax = span
    vmin = -span
    return np.clip((img2d - vmin) / (vmax - vmin), -1, 1)

In [ ]:
class Attributions(object):
    """Class for Attributions. Not efficient but for demo purposes."""
    def __init__(self, model, removetoplayer=True):
        self.model = model
        if removetoplayer: # need if model has softmax or some other layer after logits.
            self.model.layers.pop()
        self.logit_tensor = self.model.layers[-1].output[0] # get the logit tensor
        self.input_tensor = [self.model.input]
        self.grad = None
        self.compute_gradients = None

    def saliencymap(self, input_img, target_class):
        self.grad = self.model.optimizer.get_gradients(
            self.logit_tensor[target_class],
            self.input_tensor)

        self.compute_gradients = K.function(inputs=self.input_tensor,
                                            outputs=self.grad)
        saliency = self.compute_gradients([input_img])
        return saliency[0]

    def smoothgrad(self, input_img, target_class, noise_mean=0.0,
                   noise_std_spread=0.15,
                   nsamples=25):
        maxinput = np.max(input_img)
        mininput = np.min(input_img)
        stdev = noise_std_spread * (maxinput - mininput)

        total_gradients = np.zeros_like(input_img)
        for i in range(nsamples):
            noise = np.random.normal(0, stdev, input_img.shape)
            input_plus_noise = input_img + noise
            ingrad = self.saliencymap(input_plus_noise, target_class)
            total_gradients += ingrad[0]

        return total_gradients/nsamples

    def integrated_gradients(self, input_img, target_class,
                             baseline=None,
                             xsteps=25):
        if baseline is None:
            baseline = np.zeros_like(input_img)

        assert baseline.shape == input_img.shape

        x_diff = input_img - baseline
        total_gradients = np.zeros_like(input_img)
        for alpha in np.linspace(0, 1, xsteps):
            x_step = baseline + alpha * x_diff
            ingrad = self.saliencymap(x_step, target_class)
            total_gradients += ingrad[0]

        return total_gradients * x_diff/xsteps

In [ ]:
'''Make sure to run the setup script and train models before running this cell.'''
normal_model_path = '../models/epochs_40_lr_0.0001_batchsize_32_condition_normal_time_2021_07_29_12_16_39_AM_resnet50_best_freeze_weights.h5'
normal_model = load_model(normal_model_path)

In [ ]:
# process images and get predictions to prep for computing attributions
sampleimgs = ['../data/testdogimgs/beagle_14.jpg',
              '../data/testdogimgs/beagle_14_spurious.jpg']
processed_imgs = []
normal_model_prediction = []
for path in sampleimgs:
    img = load_image(path, resize=True, sztple=(224, 224))
    x = np.expand_dims(img, axis=0)
    x = preprocess_input(x)
    processed_imgs.append(x)
    preds_normal = normal_model.predict(x)
    normal_model_prediction.append(preds_normal.argmax())

In [ ]:
# setup attribution objects
attrobject_normal = Attributions(normal_model)

In [ ]:
# compute attributions for the two images for a normal model
attributions_normal_collection = []
attributions_spurious_collection = []
for ind, x in enumerate(processed_imgs):
    print(f"On img: {ind}")
    indx = normal_model_prediction[ind]
    
    print(f"Attributions for normal model.")
    sal = attrobject_normal.saliencymap(x, indx)
    sm = attrobject_normal.smoothgrad(x, indx, nsamples=10)
    ig = attrobject_normal.integrated_gradients(x, indx, xsteps=10)
    attributions_normal_collection.append((sal, sm, ig))

In [ ]:
titles = ['Beagle', 'Boxer', 'Chihuahua', 'Great Pyrenees',
          'Newfoundlands', 'Pomeranian', 'Pugs', 'Saint Bernard',
          'Wheaten Terrier', 'Yorkshire Terrier']

# Visualizing the attributions for a Normal Model

In [ ]:
# collect items to plot for img 1. Doing this manually for pedagogical reasons.
img_1_prediction = normal_model_prediction[0]
sal_img_1 = attributions_normal_collection[0][0]
sm_img_1  = attributions_normal_collection[0][1]
ig_img_1 = attributions_normal_collection[0][2]
img_1 = load_image(sampleimgs[0], resize=True, sztple=(224, 224))

img_2_prediction = normal_model_prediction[1]
sal_img_2 = attributions_normal_collection[1][0]
sm_img_2  = attributions_normal_collection[1][1]
ig_img_2 = attributions_normal_collection[1][2]
img_2 = load_image(sampleimgs[1], resize=True, sztple=(224, 224))

fig, axs = plt.subplots(2, 4)
axs[0][0].imshow(np.squeeze(img_1))
axs[0][1].imshow(grayscalenorm(np.squeeze(sal_img_1)), cmap='gray')
axs[0][2].imshow(grayscalenorm(np.squeeze(sm_img_1)), cmap = 'gray')
axs[0][3].imshow(grayscalenorm(np.squeeze(ig_img_1)), cmap='gray')

axs[1][0].imshow(np.squeeze(img_2))
axs[1][1].imshow(grayscalenorm(np.squeeze(sal_img_2)), cmap='gray')
axs[1][2].imshow(grayscalenorm(np.squeeze(sm_img_2)), cmap = 'gray')
axs[1][3].imshow(grayscalenorm(np.squeeze(ig_img_2)), cmap='gray')
# axs[4].imshow(newsalposnegnorm33, vmin=0.0, vmax=1.0, cmap='bwr')


axs[0][0].set_title("Normal Input\n"+titles[img_1_prediction], fontsize=12, fontweight='bold',
          color='red',
          bbox={'facecolor': 'black',
                'alpha': 0.8})
axs[1][0].set_title("Spurious Input\n"+titles[img_2_prediction], fontsize=12, fontweight='bold',
          color='red',
          bbox={'facecolor': 'black',
                'alpha': 0.8})
axs[0][1].set_title("Model Attribution: Gradient")
axs[0][2].set_title("Model Attribution: SmoothGrad")
axs[0][3].set_title("Model Attribution: Integrated Gradients")

plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[]);  
fig.set_figheight(9)
fig.set_figwidth(18)